In [5]:
import pandas
import json
from pprint import pprint
from connect_sql_db import build_engine
from IPython.display import clear_output

In [6]:
engine = build_engine(database_name="database1",host="35.225.193.21")

# Building Business DataFrame

In [7]:
dataset = []
for line in open("../../../data/yelp_dataset/yelp_academic_dataset_business.json"):
    dataset.append(json.loads(line))

In [8]:
for dictionary in dataset:
    dictionary["categories"] = 0
    dictionary["attributes"] = 0
    dictionary["hours"] = 0

In [9]:
business_df = pandas.DataFrame(dataset)

In [10]:
business_df.drop(
    [
    "categories",
    "attributes",
    "hours"
    ],
    axis = 1,
    inplace = True
)

In [11]:
business_df.count()

business_id     209393
name            209393
address         209393
city            209393
state           209393
postal_code     209393
latitude        209393
longitude       209393
stars           209393
review_count    209393
is_open         209393
dtype: int64

# Building Attributes DataFrame

In [ ]:
dataset = []
for line in open("../../../data/yelp_dataset/yelp_academic_dataset_business.json"):
    dataset.append(json.loads(line))

In [ ]:
unique_categories = []
for i in dataset:
    if i["categories"] != None:
        for v in i["categories"].split(","):
            unique_categories.append((i["business_id"],v.strip().replace(" ","_").replace("/","_").replace("\\","_").replace("&","and").lower()))
    elif i["categories"] == None:
        unique_categories.append((i["business_id"],"None"))
        
unique_categories = list(set(unique_categories))
pprint(unique_categories[0:10])

In [ ]:
business_df.head()

In [ ]:
attributes_df = business_df[["business_id"]].copy()

In [ ]:
columns = list(set([i[1]for i in unique_categories]))
counter=1
for i in columns:
    print(f"{counter/len(columns)}% complete {counter} of {len(columns)} columns created name: {i}")
    attributes_df[i] = 0
    counter+=1

In [ ]:
counter=1
clear_me = 0
for i in unique_categories:
    print(f"{counter/len(unique_categories)}% complete: cells affected {counter} of {len(unique_categories)}")
    attributes_df.loc[attributes_df.business_id == i[0],i[1]] = 1
    counter+=1
    clear_me += 1
    if clear_me == 50 and counter != len(unique_categories):
        clear_output()
        clear_me = 0

In [ ]:
attributes_df.head()

In [ ]:
attributes_df.count()

In [ ]:
attributes_df.to_csv("../../../data/attributes_table.csv",index=False)

# Attributes DataFrame pt. 2

In [2]:
dataset = []
for line in open("../../../data/yelp_dataset/yelp_academic_dataset_business.json"):
    dataset.append(json.loads(line))

In [3]:
attributes_df = pandas.read_csv("../../../data/attributes.csv")

In [4]:
unique_attributes = []
for i in dataset:
    if i["attributes"] != None:
        for v in dict(i["attributes"]).keys():
            unique_attributes.append(v)

unique_attributes = list(set(unique_attributes))
unique_attributes

['BusinessAcceptsBitcoin',
 'RestaurantsAttire',
 'BusinessParking',
 'Smoking',
 'GoodForDancing',
 'Corkage',
 'RestaurantsDelivery',
 'HasTV',
 'RestaurantsReservations',
 'Ambience',
 'Open24Hours',
 'Music',
 'Caters',
 'DietaryRestrictions',
 'DogsAllowed',
 'BYOB',
 'AcceptsInsurance',
 'HairSpecializesIn',
 'BusinessAcceptsCreditCards',
 'RestaurantsTableService',
 'RestaurantsCounterService',
 'CoatCheck',
 'Alcohol',
 'WheelchairAccessible',
 'RestaurantsTakeOut',
 'GoodForKids',
 'RestaurantsPriceRange2',
 'AgesAllowed',
 'NoiseLevel',
 'WiFi',
 'DriveThru',
 'GoodForMeal',
 'OutdoorSeating',
 'HappyHour',
 'BikeParking',
 'RestaurantsGoodForGroups',
 'BYOBCorkage',
 'BestNights',
 'ByAppointmentOnly']

In [5]:
for i in unique_attributes:
    attributes_df[i] = 0

In [6]:
bool_list = []
for i in dataset:
    if i["attributes"] != None:
        for v,x in dict(i["attributes"]).items():
            if "{" in x or "}" in x:
                bool_list.append((i["business_id"],v,"dict_col"))
            else:
                bool_list.append((i["business_id"],v,x.strip().replace("u'","").replace("'","").replace(" ","_").replace("/","_").replace("\\","_").replace("&","and").lower()))
    elif i["attributes"] == None:
        bool_list.append((i["business_id"],v,"None"))
    
bool_list[:10]   

[('f9NumwFMBDn751xgFiRbNA', 'BusinessAcceptsCreditCards', 'true'),
 ('f9NumwFMBDn751xgFiRbNA', 'BikeParking', 'true'),
 ('f9NumwFMBDn751xgFiRbNA', 'GoodForKids', 'false'),
 ('f9NumwFMBDn751xgFiRbNA', 'BusinessParking', 'dict_col'),
 ('f9NumwFMBDn751xgFiRbNA', 'ByAppointmentOnly', 'false'),
 ('f9NumwFMBDn751xgFiRbNA', 'RestaurantsPriceRange2', '3'),
 ('Yzvjg0SayhoZgCljUJRF9Q', 'GoodForKids', 'true'),
 ('Yzvjg0SayhoZgCljUJRF9Q', 'ByAppointmentOnly', 'true'),
 ('XNoUzKckATkOD1hP6vghZg', 'ByAppointmentOnly', 'None'),
 ('6OAZjbxqM5ol29BuHsil3w', 'BusinessAcceptsCreditCards', 'true')]

In [8]:
counter = 1
clear_me = 0
for i,v,x in bool_list:
    print(f"{counter/len(bool_list)}% complete: cell {counter} of {len(bool_list)}")
    attributes_df.loc[attributes_df.business_id == i, v] = x
    counter+=1
    clear_me += 1
    if clear_me == 50 and counter != len(bool_list):
        clear_output()
        clear_me = 0
    

0.9999839466638376% complete: cell 1432701 of 1432724
0.999984644634975% complete: cell 1432702 of 1432724
0.9999853426061126% complete: cell 1432703 of 1432724
0.99998604057725% complete: cell 1432704 of 1432724
0.9999867385483876% complete: cell 1432705 of 1432724
0.999987436519525% complete: cell 1432706 of 1432724
0.9999881344906626% complete: cell 1432707 of 1432724
0.9999888324618% complete: cell 1432708 of 1432724
0.9999895304329376% complete: cell 1432709 of 1432724
0.999990228404075% complete: cell 1432710 of 1432724
0.9999909263752126% complete: cell 1432711 of 1432724
0.99999162434635% complete: cell 1432712 of 1432724
0.9999923223174876% complete: cell 1432713 of 1432724
0.999993020288625% complete: cell 1432714 of 1432724
0.9999937182597626% complete: cell 1432715 of 1432724
0.9999944162309% complete: cell 1432716 of 1432724
0.9999951142020376% complete: cell 1432717 of 1432724
0.999995812173175% complete: cell 1432718 of 1432724
0.9999965101443126% complete: cell 1432719 

In [9]:
attributes_df.to_csv("../../../data/attributes_table_pt2.csv",index=False)

In [10]:
attributes_df.count()

business_id                 209393
pathologists                209393
adult_entertainment         209393
emergency_rooms             209393
sports_clubs                209393
                             ...  
BikeParking                 209393
RestaurantsGoodForGroups    209393
BYOBCorkage                 209393
BestNights                  209393
ByAppointmentOnly           209393
Length: 1377, dtype: int64

In [20]:
business_df.count()

business_id     209393
name            209393
address         209393
city            209393
state           209393
postal_code     209393
latitude        209393
longitude       209393
stars           209393
review_count    209393
is_open         209393
dtype: int64

In [16]:



final_table = pandas.merge(business_df,attributes_df,how="left",on="business_id")

In [32]:
for column in final_table.columns:
    print(final_table[column].value_counts())

_y9y4LNj18apmmkH-5tDqA    1
-AFMB0nVx0kVlbi73ir0Tw    1
rVvinb1r12pHKddVU6YcAQ    1
Unx0oA8MgQfmJgE7wxIEbA    1
Z4d3tYI7lb1vgiMNk9AiLg    1
                         ..
SaiduclstsnWUV7K7LDfsg    1
lDShrtVt5Iznm8HSZ-7AKg    1
bnQtcvhw3GXdLtCSQtqMeQ    1
AKMvsVr_9meoSmawkEBe4g    1
PsN2-pr48zL5Hnw85NN2eQ    1
Name: business_id, Length: 209393, dtype: int64
Starbucks                                                1184
McDonald's                                                854
Subway Restaurants                                        613
Tim Hortons                                               388
Burger King                                               337
                                                         ... 
Pasha Cafe Lounge                                           1
Avon Cafe & Coffee House                                    1
Summerlin Entrada Apartment Homes by ConAm Management       1
Jackalope Lakeside                                          1
Sweet Lee's             

0    208470
1       923
Name: hardware_stores, dtype: int64
0    209390
1         3
Name: municipality, dtype: int64
0    209383
1        10
Name: private_jet_charter, dtype: int64
0    209271
1       122
Name: fireplace_services, dtype: int64
0    209290
1       103
Name: immigration_law, dtype: int64
0    202958
1      6435
Name: real_estate, dtype: int64
0    209180
1       213
Name: television_service_providers, dtype: int64
0    208880
1       513
Name: personal_injury_law, dtype: int64
0    207983
1      1410
Name: optometrists, dtype: int64
0    209036
1       357
Name: criminal_defense_law, dtype: int64
0    209392
1         1
Name: oaxacan, dtype: int64
0    209387
1         6
Name: foot_care, dtype: int64
0    209392
1         1
Name: beach_volleyball, dtype: int64
0    208982
1       411
Name: boot_camps, dtype: int64
0    209325
1        68
Name: community_centers, dtype: int64
0    209312
1        81
Name: hypnosis_hypnotherapy, dtype: int64
0    209380
1        13
Name: s

0    209391
1         2
Name: drive-thru_bars, dtype: int64
0    209390
1         3
Name: water_suppliers, dtype: int64
0    209275
1       118
Name: cooking_schools, dtype: int64
0    209391
1         2
Name: pita, dtype: int64
0    208758
1       635
Name: breweries, dtype: int64
0    209291
1       102
Name: cycling_classes, dtype: int64
0    209390
1         3
Name: art_installation, dtype: int64
0    209378
1        15
Name: racing_experience, dtype: int64
0    209317
1        76
Name: donairs, dtype: int64
0    205792
1      3601
Name: cafes, dtype: int64
0    209337
1        56
Name: bartenders, dtype: int64
0    209392
1         1
Name: churros, dtype: int64
0    194673
1     14720
Name: automotive, dtype: int64
0    209255
1       138
Name: business_consulting, dtype: int64
0    209315
1        78
Name: malaysian, dtype: int64
0    209383
1        10
Name: well_drilling, dtype: int64
0    209262
1       131
Name: septic_services, dtype: int64
0    209370
1        23
Name: tiki

0    209210
1       183
Name: cosmetology_schools, dtype: int64
0    209113
1       280
Name: taxis, dtype: int64
0    209385
1         8
Name: honduran, dtype: int64
0    209383
1        10
Name: game_truck_rental, dtype: int64
0    209263
1       130
Name: gymnastics, dtype: int64
0    209360
1        33
Name: foundation_repair, dtype: int64
0    209327
1        66
Name: magicians, dtype: int64
0    205531
1      3862
Name: grocery, dtype: int64
0    209062
1       331
Name: playgrounds, dtype: int64
0    209244
1       149
Name: waffles, dtype: int64
0    209285
1       108
Name: special_education, dtype: int64
0    209392
1         1
Name: court_reporters, dtype: int64
0    209327
1        66
Name: boudoir_photography, dtype: int64
0    209371
1        22
Name: gemstones_and_minerals, dtype: int64
0    208832
1       561
Name: reflexology, dtype: int64
0    209386
1         7
Name: ranches, dtype: int64
0    207901
1      1492
Name: local_flavor, dtype: int64
0    209329
1        6

0    209118
1       275
Name: air_duct_cleaning, dtype: int64
0    209209
1       184
Name: gay_bars, dtype: int64
0    207072
1      2321
Name: cosmetic_dentists, dtype: int64
0    209161
1       232
Name: floral_designers, dtype: int64
0    209377
1        16
Name: wine_tasting_classes, dtype: int64
0    208247
1      1146
Name: music_venues, dtype: int64
0    209253
1       140
Name: acai_bowls, dtype: int64
0    209391
1         2
Name: bocce_ball, dtype: int64
0    209233
1       160
Name: occupational_therapy, dtype: int64
0    209037
1       356
Name: custom_cakes, dtype: int64
0    208843
1       550
Name: naturopathic_holistic, dtype: int64
0    209392
1         1
Name: rodeo, dtype: int64
0    209391
1         2
Name: bike_parking, dtype: int64
0    209390
1         3
Name: waldorf_schools, dtype: int64
0    209377
1        16
Name: club_crawl, dtype: int64
0    209391
1         2
Name: polynesian, dtype: int64
0    209362
1        31
Name: brewing_supplies, dtype: int64
0   

0    209390
1         3
Name: sledding, dtype: int64
0    208941
1       452
Name: party_supplies, dtype: int64
0    209390
1         3
Name: nicaraguan, dtype: int64
0    209392
1         1
Name: entertainment_law, dtype: int64
0    209377
1        16
Name: food_banks, dtype: int64
0    209301
1        92
Name: appraisal_services, dtype: int64
0    208027
1      1366
Name: pet_sitting, dtype: int64
0    209279
1       114
Name: snow_removal, dtype: int64
0    208723
1       670
Name: mobile_phone_accessories, dtype: int64
0    209389
1         4
Name: ski_schools, dtype: int64
0    209355
1        38
Name: roof_inspectors, dtype: int64
0    209348
1        45
Name: pet_cremation_services, dtype: int64
0    209286
1       107
Name: hair_loss_centers, dtype: int64
0    209392
1         1
Name: tonkatsu, dtype: int64
0    209357
1        36
Name: spine_surgeons, dtype: int64
0    209346
1        47
Name: trailer_dealers, dtype: int64
0    209326
1        67
Name: whiskey_bars, dtype: int

0    209193
1       200
Name: internet_cafes, dtype: int64
0    208546
1       847
Name: art_galleries, dtype: int64
0    207067
1      2326
Name: home_decor, dtype: int64
0    209339
1        54
Name: traffic_ticketing_law, dtype: int64
0    209128
1       265
Name: art_classes, dtype: int64
0    209364
1        29
Name: clock_repair, dtype: int64
0    206359
1      3034
Name: ice_cream_and_frozen_yogurt, dtype: int64
0    209295
1        98
Name: brazilian, dtype: int64
0    208260
1      1133
Name: transportation, dtype: int64
0    209336
1        57
Name: health_coach, dtype: int64
0    208745
1       648
Name: bagels, dtype: int64
0    209357
1        36
Name: currency_exchange, dtype: int64
0    208777
1       616
Name: gastropubs, dtype: int64
0    209174
1       219
Name: musical_instrument_services, dtype: int64
0    209120
1       273
Name: diagnostic_imaging, dtype: int64
0    207381
1      2012
Name: mediterranean, dtype: int64
0    209339
1        54
Name: mobile_dent_repa

0    208873
1       520
Name: carpeting, dtype: int64
0    209341
1        52
Name: lighting_stores, dtype: int64
0    209139
1       254
Name: water_purification_services, dtype: int64
0    209384
1         9
Name: qi_gong, dtype: int64
0    209380
1        13
Name: marinas, dtype: int64
0    209353
1        40
Name: tai_chi, dtype: int64
0    209388
1         5
Name: pub_food, dtype: int64
0    208591
1       802
Name: used, dtype: int64
0    209392
1         1
Name: general_festivals, dtype: int64
0    209382
1        11
Name: spin_classes, dtype: int64
0    209377
1        16
Name: fur_clothing, dtype: int64
0    208879
1       514
Name: travel_services, dtype: int64
0    209345
1        48
Name: anesthesiologists, dtype: int64
0    206640
1      2753
Name: seafood, dtype: int64
0    208577
1       816
Name: acupuncture, dtype: int64
0    202591
1      6802
Name: arts_and_entertainment, dtype: int64
0    209370
1        23
Name: holiday_decorating_services, dtype: int64
0    208148

0        192259
false     15995
None        602
true        533
none          4
Name: BusinessAcceptsBitcoin, dtype: int64
0         159540
casual     47999
dressy      1384
None         286
formal       125
none          59
Name: RestaurantsAttire, dtype: int64
dict_col    112241
0            90972
None          3206
none          2974
Name: BusinessParking, dtype: int64
0          205086
no           2230
outdoor      1619
yes           403
None           37
none           18
Name: Smoking, dtype: int64
0        204167
false      4207
true        973
None         40
none          6
Name: GoodForDancing, dtype: int64
0        208294
false       833
true        253
None          9
none          4
Name: Corkage, dtype: int64
0        151826
false     34531
true      21214
none        934
None        888
Name: RestaurantsDelivery, dtype: int64
0        154861
true      40233
false     13105
None       1144
none         50
Name: HasTV, dtype: int64
0        153587
false     33394
true    

In [36]:
for column in attributes_df.columns:
    print(column)

business_id
pathologists
adult_entertainment
emergency_rooms
sports_clubs
haitian
radio_stations
interlock_systems
musicians
indian
utilities
kiosk
formal_wear
resorts
concept_shops
medical_foot_care
reptile_shops
oxygen_bars
empanadas
internet_service_providers
music_and_video
pickleball
hybrid_car_repair
sandblasting
leisure_centers
tobacco_shops
pediatric_dentists
furniture_reupholstery
kids_activities
vocational_and_technical_school
fingerprinting
horse_racing
carpet_dyeing
noodles
storefront_clinics
knitting_supplies
mailbox_centers
medical_spas
home_organization
workers_compensation_law
childbirth_education
hot_air_balloons
bike_repair_maintenance
mongolian
pet_boarding
sports_betting
books
video_game_stores
videos_and_video_game_rental
pop-up_restaurants
screen_printing_t-shirt_printing
day_camps
couriers_and_delivery_services
flyboarding
session_photography
windows_installation
boat_repair
addiction_medicine
police_departments
skate_parks
architectural_tours
cultural_center
atv

In [ ]:
[
    "pathologists",
    "emergency_rooms",
    "radio_stations",
    "interlock_systems",
    "kiosk",
    "concept_shops",
    "medical_foot_care",
    "reptile_shops",
    "oxygen_bars"
]

In [44]:
attributes_df.oxygen_bars.value_counts()

0    209373
1        20
Name: oxygen_bars, dtype: int64

In [21]:
final_table.to_csv("~/Desktop/merged_table.csv", index = False)

In [20]:
final_table.columns = [i.lower().replace(" ","_").replace("&","and").replace("(","").replace(")","") for i in final_table.columns]


In [46]:
engine = build_engine(database_name="database1",host="35.225.193.21")

In [ ]:
final_table.to_sql(
    "merged_table",
    con = engine,
    index = False, 
    method = 'multi',
    chunksize = 10000,
    if_exists="replace"
)

In [22]:
len(final_table.columns)

1387